In [1]:
import pandas as pd

df = pd.read_csv("data/train.csv")
df = pd.read_csv("data/train.csv")
df = df.dropna()
X1 = df["question1"].values
X2 = df["question2"].values
y = df["is_duplicate"].values
X= [X1, X2]

In [12]:
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.seq2seq as seq2seq
from hedgeable_ai.models.nn import BaseModel, get_shape, get_length

import tensorflow as tf

from hedgeable_ai.models.nn.params import nn_is_logit
from hedgeable_ai.models.nn import MultiNN, get_shape



class DialogueAgent(MultiNN):
    def __init__(self, maxlen, *args, **kwargs):
        self.embedding_size = 300
        self.vocabulary_size = 100
        self.maxlen = maxlen
        super().__init__(*args, **kwargs)
   
        
    def _build_graph(self):
        """Build tensorflow graph
        
        Note:
            You build graphs for output and input, which will be used 
            for training and prediction.
        """
        self.encoder_input = tf.placeholder(tf.int32, shape=(None, self.maxlen), name="encoder_input")
        self.decoder_input = tf.placeholder(tf.int32, shape=(None, self.maxlen), name="decoder_input")
        self.target = tf.placeholder(tf.int32, shape=(None,self.maxlen), name="target")
        word_embeddings = tf.get_variable("word_embeddings", [self.vocabulary_size, self.embedding_size])
        embedded_encoder = tf.gather(word_embeddings, self.encoder_input)
        embedded_decoder = tf.gather(word_embeddings, self.decoder_input)
        #  Build Seq2Seq Model
        cell = [rnn.LSTMCell(512) for _ in range(3)]
        cell = rnn.MultiRNNCell(cell)
        length = get_length(self.encoder_input)
        outputs, state = tf.nn.dynamic_rnn(cell,embedded_encoder, dtype=tf.float32)
        helper = seq2seq.TrainingHelper(embedded_decoder, sequence_length=(20, 20), time_major=False)
        decoder =  seq2seq.BasicDecoder(cell, helper, state)
        final_outputs, final_state, final_sequence_lengths\
            = seq2seq.dynamic_decode(decoder, output_time_major=False,
                                     impute_finished=True, maximum_iterations=None,
                                     parallel_iterations=32, swap_memory=False, scope=None)
        weights = tf.ones_like(self.target)
        self.loss = seq2seq.sequence_loss(final_outputs, self.target, weights)
        
        self.learning_rate_op = self._get_learning_rate()
        self.train_step =\
            tf.train.AdamOptimizer(self.learning_rate_op).minimize(self.loss)
        # Build tensorboad graph
        with tf.name_scope("summary"):
            self._build_summaries()
        
        # initialize graph
        self.sess.run(tf.global_variables_initializer())
        
    def _optimize(self, batch_X, batch_y, *args, **kwargs):
        feed_dict={
                self.encoder_input: batch_X[0],
                self.decoder_input: batch_X[1],
                self.training: True,
                self.target: batch_y}
        _, loss = self.sess.run([self.train_step, self.loss], feed_dict=feed_dict)
        return loss

In [13]:
tf.reset_default_graph()

agent = DialogueAgent(20)

AttributeError: 'BasicDecoderOutput' object has no attribute 'get_shape'

In [ ]:
tf.ones_like

In [2]:
import tensorflow.contrib.seq2seq as seq2seq

In [3]:
seq2seq.basic_rnn_seq2seq

AttributeError: module 'tensorflow.contrib.seq2seq' has no attribute 'basic_rnn_seq2seq'